# Choice Set space for Neural Data

 The idea is to create a choice set that: samples the same SV difference (+ - some variation) with different trials  

In [2]:
#Import the relevant libraries
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
pd.options.display.max_rows = 999

# 1. Input: patient's alpha and beta parameters

In [5]:
#Define files & data pathway
patient_code = 'patient1_calibration'
patient_split_dir = '/Volumes/UCDN/Santiago_Guardo/data/split'
utility_dir = '/Volumes/UCDN/Santiago_Guardo/data/utility'
utility_analysis = '/Volumes/UCDN/Santiago_Guardo/data/utility/split_CRDM_analysis.csv'
parameters = pd.read_csv(utility_analysis)


In [10]:

#Define the parameter values for the given participant. 
filtered_parameters = parameters[parameters['subject'] == patient_code]

# Create alpha_pos and alpha_neg variables
alpha_pos = filtered_parameters.loc[filtered_parameters['domain'] == 'gain', 'alpha'].values[0]
alpha_neg = filtered_parameters.loc[filtered_parameters['domain'] == 'loss', 'alpha'].values[0]

beta_pos = filtered_parameters.loc[filtered_parameters['domain'] == 'gain', 'beta'].values[0]
beta_neg = filtered_parameters.loc[filtered_parameters['domain'] == 'loss', 'beta'].values[0]

print("alpha_pos:", alpha_pos)
print("alpha_neg:", alpha_neg)

print("beta_pos:", beta_pos)
print("beta_pos:", beta_neg)



alpha_pos: 0.1286154715090123
alpha_neg: 0.415047662498603
beta_pos: -0.3918035142034978
beta_pos: 4.167
